## Example 2: Exploring StateGraph in depth (State, Node, Edges)

Reference: [Getting Started with LangGraph: A Beginner’s Guide to Building Intelligent Workflows](https://medium.com/@ashutoshsharmaengg/getting-started-with-langgraph-a-beginners-guide-to-building-intelligent-workflows-67eeee0899d0)

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from dotenv import load_dotenv
import os
import random

# This loads the .env file for loading api-key
load_dotenv() 

True

The State schema is the input for all nodes and edges. It is the shared memory passed between the nodes. Each node can read from the state and update it.

In [2]:
# A single key to hold the State (string type)
class State(TypedDict): 
    graph_state: str 

In a StateGraph, nodes receive the entire current state as their first positional argument. Their return value is used to update the state. 

By default, if a node returns `{'key': value}`, it will override the existing key in the state.

We define 3 nodes in the StateGraph.

In [3]:
def node_1(state):
    print('--Node1--')
    # Append to the graph_state
    return {'graph_state': state['graph_state'] + "I am taking the CS 7295 class this fall. I am"}

def node_2(state):
    print('--Node2--')
    return {'graph_state': state['graph_state'] + " excited!"}

def node_3(state):
    print('--Node3--')
    return {'graph_state': state['graph_state'] + " nervous!"}

- Directed Edges sequentially move between nodes
- Conditional Edges decides path based on outputs of the nodes

**Important:** Nodes update the state, while conditional edge functions decide which node to go to next based on the state.

In [4]:
# Function to route conditional edge
def mood(state) -> Literal["node_2", "node_3"]:
    user_input = state['graph_state']    # Accessing the current state
    if random.random() > 0.5:            # 50% of the time we will return node2          
        return 'call node_2'             # This string matches a key in the conditional_edges mapping   
    return 'call node_3'                 # 50% of the time we will return node3

### Build graph

1. Initialize StateGraph: pass your State class to it
2. Add nodes (functions as nodes)
3. Define flow with Edges: Connect your nodes
    - **add_edge** for direct movement between nodes
    - **add_conditional_edges** for decision points
4. Set START and END points
5. Compile graph

In [5]:
# Initialize with our defined State
workflow = StateGraph(State)         

# Add nodes
workflow.add_node('node_1',node_1)
workflow.add_node('node_2',node_2)
workflow.add_node('node_3',node_3)

# Define the flow
workflow.add_edge(START,'node_1')      # Start at node_1
workflow.add_conditional_edges(
    'node_1',                          # From node_1
    mood,                              # Use mood function to decide
    {
        'call node_2':'node_2',        # If mood returns 'call node_2', go to node_2
        'call node_3':'node_3'         # If mood returns 'call node_3', go to node_3
    }
)
workflow.add_edge('node_2',END)        # From node_2, go to END
workflow.add_edge('node_3',END)        # From node_3, go to END
app = workflow.compile()               # Compile the graph

`invoke()` runs the graph from the START node, and moved through the defined nodes and edges. Once, all nodes are executed, the final state is returned. 

In [6]:
# Test the workflow
initial_state = {"graph_state": "Hi! "}
result = app.invoke(initial_state)

print("\n--- Final Result ---")
print(result['graph_state'])

--Node1--
--Node2--

--- Final Result ---
Hi! I am taking the CS 7295 class this fall. I am excited!


Running the workflow multiple times to see different outcomes.

In [7]:
print("=== Testing the workflow ===\n")

for i in range(3):
    print(f"--- Run {i+1} ---")
    initial_state = {"graph_state": ""}
    result = app.invoke(initial_state)
    print(f"Final output: {result['graph_state']}")
    print()

=== Testing the workflow ===

--- Run 1 ---
--Node1--
--Node2--
Final output: I am taking the CS 7295 class this fall. I am excited!

--- Run 2 ---
--Node1--
--Node2--
Final output: I am taking the CS 7295 class this fall. I am excited!

--- Run 3 ---
--Node1--
--Node2--
Final output: I am taking the CS 7295 class this fall. I am excited!

